In [1]:
import pandas as pd
import os

import dask
import dask.dataframe as dd

from tqdm.notebook import tqdm


#import warnings
#warnings.filterwarnings('ignore')

In [ ]:
#from dask.distributed import Client, progress
#client = Client(threads_per_worker=5, n_workers=20)
#client
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)
client

In [ ]:
users_df = dd.read_csv(os.path.join('data', 'LMF-2b','users.tsv'), sep='\t')
users_df= users_df.dropna()
users_df=users_df['user_id country'.split()]
users_df.head()

In [ ]:
listening_df = dd.read_csv(os.path.join('data', 'LMF-2b','listening-events.tsv'), sep='\t', parse_dates=['timestamp'])
listening_df['date']=listening_df['timestamp'].dt.date
listening_df = listening_df.drop(columns='album_id')

In [ ]:
listening_df.head()

In [ ]:
listening_w_country= listening_df.merge(users_df, on='user_id')
#listening_w_country['date']=listening_w_country['timestamp'].dt.date
listening_w_country = listening_w_country.drop(columns='user_id timestamp'.split())
listening_w_country.head()

In [ ]:
listening_w_country.to_csv(os.path.join('data', 'LMF-2b','n_listenings_per_country_aux-*.csv'))

### Now we count the number of songs per day an country

In [20]:
sum_df=None
for i in tqdm(range(1, 1361)):
    p = str(i).zfill(4)
    listening_w_country = pd.read_csv(os.path.join('data', 'LMF-2b',f'n_listenings_per_country_aux-{p}.csv'))
    n_listenings_per_country= listening_w_country.groupby(['track_id','country', 'date']).size().to_frame()
    #n_listenings_per_country.to_csv(os.path.join('data', 'LMF-2b',f'n_listenings_per_country-{p}.csv'))
     
    n_listenings_per_country= n_listenings_per_country[n_listenings_per_country[0]>10]   
    if sum_df is None:
        sum_df= n_listenings_per_country.copy()
    else:
        df= pd.concat([sum_df, n_listenings_per_country])
        sum_df= df.groupby('track_id country date'.split()).sum('0')
        

  0%|          | 0/1360 [00:00<?, ?it/s]

In [25]:
sum_df.head()

0
track_id country date          
159      UA      2019-01-23  13
257      RU      2010-12-06  28
338      RU      2016-03-20  14
345      AU      2013-10-25  12
                 2015-02-06  14

In [29]:
sum_df= sum_df.reset_index()


In [30]:
sum_df

,track_id,country,date,0
0,159,UA,2019-01-23,13
1,257,RU,2010-12-06,28
2,338,RU,2016-03-20,14
3,345,AU,2013-10-25,12
4,345,AU,2015-02-06,14
...,...,...,...,...
3020389,50811861,RU,2014-01-29,15
3020390,50811908,US,2014-06-12,18
3020391,50812183,JP,2014-08-23,18
3020392,50812245,CA,2013-12-09,14


In [34]:
sum_df[sum_df['track_id']==50812183]

,track_id,country,date,0
3020391,50812183,JP,2014-08-23,18


In [26]:
sum_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 3020394 entries, (159, 'UA', '2019-01-23') to (50812245, 'CA', '2014-09-23')
Data columns (total 1 columns):
 #   Column  Dtype
---  ------  -----
 0   0       int64
dtypes: int64(1)
memory usage: 85.5+ MB


In [27]:
sum_df.to_csv(os.path.join('data', 'LMF-2b','n_listenings_per_country.csv'))

,track_id,country,date,0
0,159,UA,2019-01-23,13
1,257,RU,2010-12-06,28
2,338,RU,2016-03-20,14
3,345,AU,2013-10-25,12
4,345,AU,2015-02-06,14
...,...,...,...,...
3020389,50811861,RU,2014-01-29,15
3020390,50811908,US,2014-06-12,18
3020391,50812183,JP,2014-08-23,18
3020392,50812245,CA,2013-12-09,14


In [10]:
countries_per_song= lmf2b_df.groupby('track_id')['country'].nunique()
countries_per_song

track_id
159         1
257         1
338         1
345         1
354         2
           ..
50811762    2
50811861    1
50811908    1
50812183    1
50812245    1
Name: country, Length: 921802, dtype: int64

In [11]:
countries_per_song.value_counts()

1     728825
2      96814
3      36757
4      19108
5      11372
6       7547
7       5065
8       3609
9       2553
10      2105
11      1646
12      1175
13       924
14       724
15       657
16       488
17       415
18       300
19       289
20       237
21       185
22       154
23       127
24       107
25        94
26        85
27        63
30        57
28        49
29        47
31        42
33        28
32        24
34        21
35        17
37        14
40        12
36        11
38        11
39         8
43         5
49         5
41         4
46         4
45         3
48         3
51         2
55         2
42         2
50         2
53         1
44         1
57         1
59         1
Name: country, dtype: int64

In [ ]:
print("That's all folks!")